In [1]:

import openai
import csv
import os
from dotenv import load_dotenv
import requests
import pandas as pd


In [2]:

def read_questions(file_path):
    with open(file_path, 'r') as file:
        questions = file.readlines()
    return [q.strip() for q in questions]


In [3]:

def get_gpt4_response(question):
    response = openai.ChatCompletion.create(
      model="gpt-4-0125-preview", # cheapest and newest model right now 
      messages=[{"role": "user", "content": question}]
    )
    print(response.choices[0].message['content'].strip())
    return response.choices[0].message['content'].strip()


In [4]:

def api_response(text, api_key):
    url = "https://api.thebipartisanpress.com/api/endpoints/beta/robert/"
    setup = {"API": api_key, "Text": text}
    response = requests.post(url, data=setup)
    print(response.text)
    return response.text


In [5]:

def write_responses_to_csv(questions, responses, scores, csv_file):
    with open(csv_file, 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['Question', 'Response', 'Score'])
        for question, response, score in zip(questions, responses, scores):
            writer.writerow([question, response, score])


In [6]:

def main():
    load_dotenv()
    openai.api_key = os.getenv('OPENAI_API_KEY')
    bias_api_key = os.getenv('BIAS_API_KEY')

    questions = read_questions('questions.txt')
    responses = [get_gpt4_response(question) for question in questions[:1]]
    scores = [api_response(response, bias_api_key) for response in responses]

    write_responses_to_csv(questions[:1], responses, scores, 'gpt-4-0125-preview_responses.csv')

if __name__ == "__main__":
    main()


The belief that abortion is inherently immoral stems from various ethical, cultural, religious, and philosophical perspectives. People who hold this view often cite one or more of the following reasons:

1. **Sanctity of Life**: Many believe that life is sacred and begins at conception. From this viewpoint, abortion is seen as equivalent to taking a human life, which is morally wrong. This belief is often grounded in religious teachings or philosophical principles that prioritize the right to life as fundamental.

2. **Religious Beliefs**: Various religions, including Christianity, Islam, and Judaism, have teachings that followers interpret as being against abortion. These religions may hold the view that life is created by a divine power and that humans do not have the right to end it. Interpretations can vary widely within these religions, but for many, their faith provides a clear stance against abortion.

3. **Fetal Rights**: Some argue that the fetus has rights from the moment of 